## Creating Spark Session

In [6]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark=SparkSession.builder.appName("Practice-1").getOrCreate()

In [11]:
spark.sparkContext.stop()

In [2]:
spark.version

'2.4.8'

## Creating Spark Context

In [3]:
from pyspark import SparkConf, SparkContext
conf=SparkConf()
conf.setMaster('local')
conf.setAppName("Practice-2")
sc=SparkContext.getOrCreate(conf=conf)

In [4]:
sc.version

'2.4.8'

In [5]:
sc.defaultParallelism

12

# PySpark Job

In [6]:
def mod(x):
    return (x,x%2)

In [7]:
rdd=sc.parallelize(range(1,10))

In [8]:
old_rdd=(id(rdd))
print(old_rdd)

2404663910344


In [9]:
rdd=rdd.map(mod)

In [10]:
new_rdd=(id(rdd))
print(new_rdd)

2404286265480


In [11]:
import ctypes
print (ctypes.cast(old_rdd, ctypes.py_object).value)

PythonRDD[1] at RDD at PythonRDD.scala:53


In [12]:
import ctypes
print (ctypes.cast(new_rdd, ctypes.py_object).value)

PythonRDD[2] at RDD at PythonRDD.scala:53


In [13]:
# import gc
# del rdd
# gc.collect()

In [14]:
rdd.take(5)

[(1, 1), (2, 0), (3, 1), (4, 0), (5, 1)]

In [13]:
rdd.collect()

[(1, 1), (2, 0), (3, 1), (4, 0), (5, 1), (6, 0), (7, 1), (8, 0), (9, 1)]

In [14]:
rdd.glom().collect()

[[(1, 1), (2, 0), (3, 1), (4, 0), (5, 1), (6, 0), (7, 1), (8, 0), (9, 1)]]

In [19]:
len(rdd.glom().collect())

1

In [33]:
rdd=sc.parallelize(range(1,10),3)
rdd.glom().collect()

[[1, 2, 3], [4, 5, 6], [7, 8, 9]]

In [34]:
rdd=rdd.repartition(2)
rdd.glom().collect()

[[1, 2, 3, 7, 8, 9], [4, 5, 6]]

In [35]:
rdd=rdd.repartition(5)
rdd.glom().collect()

[[], [1, 2, 3], [7, 8, 9], [4, 5, 6], []]

In [36]:
rdd=rdd.repartition(1)
rdd.glom().collect()

[[1, 2, 3, 7, 8, 9, 4, 5, 6]]

## Read from text file

In [243]:
textrdd=sc.textFile("textfiles/Spark.txt")
# textrdd.take(2)
textrdd.glom().collect()

[['Transformations in Spark are “lazy”, meaning that they do not compute their results right away. Instead, they just “remember” the operation to be performed and the dataset (e.g., file) to which the operation is to be performed. The transformations are only actually computed when an action is called and the result is returned to the driver program. This design enables Spark to run more efficiently. For example, if a big file was transformed in various ways and passed to first action, Spark would only process and return the result for the first line, rather than do the work for the entire file.'],
 ['',
  'By default, each transformed RDD may be recomputed each time you run an action on it. However, you may also persist an RDD in memory using the persist or cache method, in which case Spark will keep the elements around on the cluster for much faster access the next time you query it.']]

## word count (method-1)

In [255]:
def f(lines):
    lines=lines.lower()
    lines=lines.split(' ')
    return lines
flatrdd=textrdd.flatMap(f)
# flatrdd.collect()

In [256]:
# rough demo
def fun(x):
    l=""
    for i in x:
        l=l+i
    return l
words.map(lambda x:(1,x)).groupByKey().mapValues(fun).take(5)

[(1,
  'TransformationsinSparkare“lazy”,meaningthattheydonotcomputetheirresultsrightaway.Instead,theyjust“remember”theoperationtobeperformedandthedataset(e.g.,file)towhichtheoperationistobeperformed.Thetransformationsareonlyactuallycomputedwhenanactioniscalledandtheresultisreturnedtothedriverprogram.ThisdesignenablesSparktorunmoreefficiently.Forexample,ifabigfilewastransformedinvariouswaysandpassedtofirstaction,Sparkwouldonlyprocessandreturntheresultforthefirstline,ratherthandotheworkfortheentirefile.Bydefault,eachtransformedRDDmayberecomputedeachtimeyourunanactiononit.However,youmayalsopersistanRDDinmemoryusingthepersistorcachemethod,inwhichcaseSparkwillkeeptheelementsaroundontheclusterformuchfasteraccessthenexttimeyouqueryit.')]

In [233]:
wordcount=words.map(lambda x:(x,1))
# wordcount.take(5)

In [234]:
wordcount=wordcount.groupByKey()
# wordcount.take(5)

In [235]:
wordcount=wordcount.mapValues(sum)
# wordcount.take(5)

In [236]:
wordcount=wordcount.map(lambda x:(x[1],x[0])).sortByKey(False)
wordcount.take(5)

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 317.0 failed 1 times, most recent failure: Lost task 0.0 in stage 317.0 (TID 1814, localhost, executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "C:\work\spark-2.4.8-bin-hadoop2.7\python\lib\pyspark.zip\pyspark\worker.py", line 377, in main
  File "C:\work\spark-2.4.8-bin-hadoop2.7\python\lib\pyspark.zip\pyspark\worker.py", line 372, in process
  File "C:\work\spark-2.4.8-bin-hadoop2.7\python\pyspark\rdd.py", line 2499, in pipeline_func
    return func(split, prev_func(split, iterator))
  File "C:\work\spark-2.4.8-bin-hadoop2.7\python\pyspark\rdd.py", line 2499, in pipeline_func
    return func(split, prev_func(split, iterator))
  File "C:\work\spark-2.4.8-bin-hadoop2.7\python\pyspark\rdd.py", line 2499, in pipeline_func
    return func(split, pre

## word count (method-2)

In [222]:
nonempty_lines=textrdd.filter(lambda x: len(x)>0)
nonempty_lines.take(2)

['Transformations in Spark are “lazy”, meaning that they do not compute their results right away. Instead, they just “remember” the operation to be performed and the dataset (e.g., file) to which the operation is to be performed. The transformations are only actually computed when an action is called and the result is returned to the driver program. This design enables Spark to run more efficiently. For example, if a big file was transformed in various ways and passed to first action, Spark would only process and return the result for the first line, rather than do the work for the entire file.',
 'By default, each transformed RDD may be recomputed each time you run an action on it. However, you may also persist an RDD in memory using the persist or cache method, in which case Spark will keep the elements around on the cluster for much faster access the next time you query it.']

In [223]:
words=nonempty_lines.flatMap(lambda x: x.split(' '))
words.take(5)

['Transformations', 'in', 'Spark', 'are', '“lazy”,']

In [224]:
# rough demo
words.map(lambda x:(1,x)).reduceByKey(add).take(5)

[(1,
  'TransformationsinSparkare“lazy”,meaningthattheydonotcomputetheirresultsrightaway.Instead,theyjust“remember”theoperationtobeperformedandthedataset(e.g.,file)towhichtheoperationistobeperformed.Thetransformationsareonlyactuallycomputedwhenanactioniscalledandtheresultisreturnedtothedriverprogram.ThisdesignenablesSparktorunmoreefficiently.Forexample,ifabigfilewastransformedinvariouswaysandpassedtofirstaction,Sparkwouldonlyprocessandreturntheresultforthefirstline,ratherthandotheworkfortheentirefile.Bydefault,eachtransformedRDDmayberecomputedeachtimeyourunanactiononit.However,youmayalsopersistanRDDinmemoryusingthepersistorcachemethod,inwhichcaseSparkwillkeeptheelementsaroundontheclusterformuchfasteraccessthenexttimeyouqueryit.')]

In [225]:
wordcount=words.map(lambda x:(x,1))
wordcount.take(5)

[('Transformations', 1), ('in', 1), ('Spark', 1), ('are', 1), ('“lazy”,', 1)]

In [226]:
wordcount=wordcount.reduceByKey(add)
wordcount.take(5)

[('in', 4), ('Spark', 4), ('are', 2), ('meaning', 1), ('do', 2)]

In [227]:
wordcount=wordcount.map(lambda x:(x[1],x[0])).sortByKey(False)
wordcount.take(5)

[(13, 'the'), (6, 'to'), (4, 'in'), (4, 'Spark'), (4, 'and')]

## caching

In [99]:
textrdd.cache()

textfiles/Spark.txt MapPartitionsRDD[121] at textFile at NativeMethodAccessorImpl.java:0

## wholeTextFiles

In [101]:
textrdd=sc.wholeTextFiles("textfiles")

In [102]:
textrdd.collect()

[('file:/D:/Projects/Python/pyspark/textfiles/data.txt',
  "'The online encyclopedia project, Wikipedia, is the most popular wiki-based website, and is one of the most widely viewed sites in the world, having been ranked in the top twenty since 2007.[3] "),
 ('file:/D:/Projects/Python/pyspark/textfiles/Spark.txt',
  'A wiki (/ˈwɪki/ (About this soundlisten) WIK-ee) is a hypertext publication collaboratively edited and managed by its own audience directly using a web browser. A typical wiki contains multiple pages for the subjects or scope of the project and could be either open to the public or limited to use within an organization for maintaining its internal knowledge base.\r\n\r\nWikis are enabled by wiki software, otherwise known as wiki engines. A wiki engine, being a form of a content management system, differs from other web-based systems such as blog software, in that the content is created without any defined owner or leader, and wikis have little inherent structure, allowing 

# Transformation

## map

In [89]:
x=sc.parallelize(["spark", "rdd", "example", "sample"])
y=x.map(lambda x:(x,len(x)))
y.collect()

[('spark', 5), ('rdd', 3), ('example', 7), ('sample', 6)]

## Tuple map

In [85]:
rdd = sc.parallelize([('a' ,7),('b' ,2),('c' ,2)])
rdd.collect()

[('a', 7), ('b', 2), ('c', 2)]

In [90]:
def changes(tup):
    return (tup[0],len(tup[0]))

In [91]:
rdd=rdd.map(changes)
rdd.collect()

[('a', 1), ('b', 1), ('c', 1)]

## flatMap

In [41]:
x=sc.parallelize([2,3,4])
y=x.flatMap(lambda x: range(1,x))
y.collect()

[1, 1, 2, 1, 2, 3]

## filter

In [42]:
x=sc.parallelize([1,2,3,4,5])
x.filter(lambda x: x%2==0).collect()

[2, 4]

## sample

In [28]:
parallel=sc.parallelize(range(10))
parallel=parallel.sample(True,1)
parallel.collect()

[0, 0, 2, 4, 4, 5, 5, 9]

In [29]:
parallel=sc.parallelize(range(10))
parallel=parallel.sample(True,0.5)
parallel.collect()

[1, 2, 5, 5, 8, 9]

In [31]:
parallel=sc.parallelize(range(10))
parallel=parallel.sample(True,0.2)
parallel.collect()

[8]

In [33]:
parallel=sc.parallelize(range(10))
parallel=parallel.sample(False,1)
parallel.collect()

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

In [34]:
parallel=sc.parallelize(range(10))
parallel=parallel.sample(False,0.5)
parallel.collect()

[2, 3, 5, 6, 7, 9]

In [35]:
parallel=sc.parallelize(range(10))
parallel=parallel.sample(False,0.2)
parallel.collect()

[0, 6]

In [36]:
parallel=sc.parallelize(range(10))
parallel=parallel.sample(False,0.2)
parallel.collect()

[1]

## union

In [37]:
parallel=sc.parallelize(range(1,9))
par=sc.parallelize(range(5,15))
parallel.union(par).collect()

[1, 2, 3, 4, 5, 6, 7, 8, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14]

## Intersection

In [38]:
parallel=sc.parallelize(range(1,9))
par=sc.parallelize(range(5,15))
parallel.intersection(par).collect()

[5, 6, 7, 8]

## distinct

In [39]:
parallel=sc.parallelize(range(1,9))
par=sc.parallelize(range(5,15))
parallel.union(par).distinct().collect()

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14]

## sortBy

In [40]:
y=sc.parallelize([5,7,1,3,2,1])
y.sortBy(lambda c: c,True).collect()

[1, 1, 2, 3, 5, 7]

In [43]:
y=sc.parallelize([('H',10), ('A',26), ('Z',1), ('L',5), ('A',5)])
y.sortBy(lambda c: c,False).collect()

[('Z', 1), ('L', 5), ('H', 10), ('A', 26), ('A', 5)]

## mapPartitions

In [44]:
rdd=sc.parallelize([1,2,3,4],2)
def f(itr): yield sum(itr)
rdd.mapPartitions(f).collect()

[3, 7]

## mapPartitionsWithIndex

In [54]:
rdd=sc.parallelize([1,2,3,4,5,6,7,8],4)
def f(splitIndex,itr): yield (splitIndex,sum(itr))
rdd.mapPartitionsWithIndex(f).collect()

[(0, 3), (1, 7), (2, 11), (3, 15)]

In [55]:
rdd.glom().collect()

[[1, 2], [3, 4], [5, 6], [7, 8]]

## groupBy

In [58]:
rdd=sc.parallelize([1,2,3,4,5,6,7,8])
result=rdd.groupBy(lambda x:x%2).collect()
result

[(0, <pyspark.resultiterable.ResultIterable at 0x22fe4d9f6c8>),
 (1, <pyspark.resultiterable.ResultIterable at 0x22fe4d9f708>)]

In [60]:
[(x,sorted(y)) for (x,y) in result]

[(0, [2, 4, 6, 8]), (1, [1, 3, 5, 7])]

In [64]:
x=sc.parallelize(range(0,5)).keyBy(lambda x:x*x)
x.collect()

[(0, 0), (1, 1), (4, 2), (9, 3), (16, 4)]

## zip

In [63]:
x=sc.parallelize(range(0,5))
y=sc.parallelize(range(1000,1005))
x.zip(y).collect()

[(0, 1000), (1, 1001), (2, 1002), (3, 1003), (4, 1004)]

## zip + groupBy

In [68]:
x=sc.parallelize(range(0,5)).keyBy(lambda x:x*x)
y=sc.parallelize(zip(range(0,5), range(0,5)))
[(a, list(map(list,b))) for a,b in sorted(x.cogroup(y).collect())]

[(0, [[0], [0]]),
 (1, [[1], [1]]),
 (2, [[], [2]]),
 (3, [[], [3]]),
 (4, [[2], [4]]),
 (9, [[3], []]),
 (16, [[4], []])]

## Repartition

In [206]:
rdd=sc.parallelize(range(0,9),4)
rdd.glom().collect()

[[0, 1], [2, 3], [4, 5], [6, 7, 8]]

In [207]:
rdd=rdd.repartition(2)
rdd.glom().collect()

[[0, 1, 4, 5, 6, 7, 8], [2, 3]]

## coalesce

In [77]:
rdd=sc.parallelize(range(1,6),3)
rdd.glom().collect()

[[1], [2, 3], [4, 5]]

In [78]:
rdd=rdd.coalesce(2)
rdd.glom().collect()

[[1], [2, 3, 4, 5]]

In [79]:
rdd=rdd.coalesce(3)
rdd.glom().collect()

[[1], [2, 3, 4, 5]]

# Actions

## reduce

In [83]:
def add(x,y):
    return x+y
sc.parallelize(range(6)).reduce(add)

15

## first

In [84]:
sc.parallelize(range(6)).first()

0

## takeOrdered

In [87]:
sc.parallelize([1,5,3,9,4,0,2]).takeOrdered(4)

[0, 1, 2, 3]

## take

In [88]:
sc.parallelize([1,5,3,9,4,0,2]).take(4)

[1, 5, 3, 9]

## count

In [89]:
sc.parallelize([1,5,3,9,4,0,2]).count()

7

## collect

In [90]:
sc.parallelize([1,5,2,1,4,0,2]).collect()

[1, 5, 2, 1, 4, 0, 2]

In [91]:
sc.parallelize([1,5,2,1,4,0,2]).distinct().collect()

[0, 1, 2, 4, 5]

In [92]:
alpha=sc.parallelize([(1,'a'), (2,'b'), (3,'c')])
alpha.collectAsMap()

{1: 'a', 2: 'b', 3: 'c'}

## saveAsTextFile

In [104]:
a=sc.parallelize(range(1,10000),3)
a.saveAsTextFile("output/sample1")

In [105]:
a=sc.parallelize(range(1,10),1)
a.saveAsTextFile("output/sample2")

## foreach

In [143]:
rdd=spark.sparkContext.parallelize([1,2,3,4,5])
rdd.foreach(lambda x: x+1)

## foreachPartition

In [144]:
def f(itr):
    for x in itr:
        return (x)
sc.parallelize([1,2,3,4,5]).foreachPartition(f)

## Mathematical Actions

In [151]:
numbers=sc.parallelize(range(1,100))

In [152]:
numbers.sum()

4950

In [153]:
numbers.min()

1

In [154]:
numbers.max()

99

In [155]:
numbers.variance()

816.6666666666666

In [160]:
numbers.mean()

50.0

In [157]:
numbers.stdev()

28.577380332470412

## countByValue

In [158]:
a=sc.parallelize([1,2,3,4,5,6,7,8,2,4,2,3,3,3,1,1,1])
a.countByValue()

defaultdict(int, {1: 4, 2: 3, 3: 4, 4: 2, 5: 1, 6: 1, 7: 1, 8: 1})

## toDebugString

In [162]:
a=sc.parallelize(range(1,19),3)
b=sc.parallelize(range(1,13),3)
c=a.subtract(b)
c.toDebugString()

b'(6) PythonRDD[424] at RDD at PythonRDD.scala:53 []\n |  MapPartitionsRDD[423] at mapPartitions at PythonRDD.scala:133 []\n |  ShuffledRDD[422] at partitionBy at NativeMethodAccessorImpl.java:0 []\n +-(6) PairwiseRDD[421] at subtract at C:\\Users\\DMS240~1\\AppData\\Local\\Temp/ipykernel_10552/2648738274.py:3 []\n    |  PythonRDD[420] at subtract at C:\\Users\\DMS240~1\\AppData\\Local\\Temp/ipykernel_10552/2648738274.py:3 []\n    |  UnionRDD[419] at union at NativeMethodAccessorImpl.java:0 []\n    |  PythonRDD[417] at RDD at PythonRDD.scala:53 []\n    |  ParallelCollectionRDD[415] at parallelize at PythonRDD.scala:195 []\n    |  PythonRDD[418] at RDD at PythonRDD.scala:53 []\n    |  ParallelCollectionRDD[416] at parallelize at PythonRDD.scala:195 []'

In [161]:
c.collect()

[18, 13, 14, 15, 16, 17]